# Dashboard

This pipeline processes data into a form to drive the dashboard.

The parameters are injected using papermill.

In [ ]:
DEBUG = 'q3'
if DEBUG:
    import yaml
    with open('./params.yaml') as f:
        config = yaml.load(f, yaml.SafeLoader)['dashboards'][DEBUG]

    target = config['target']
    start = config['start']
    end = config['end']
    fixed_date = config['fixed_date']

In [ ]:
target: str
start: str
end: str
fixed_date: str

## Setup

Import some libraries

In [ ]:
import json
from datetime import date
from ast import literal_eval

import petl as etl
from utils.paths import PUBLISHED, SITE
from utils.themes.programme_slice import ProgrammeSlice

Set the target directory, and ensure it exists

In [ ]:
TARGET = SITE / 'insights/dashboard' / target / '_data'
TARGET.mkdir(exist_ok=True, parents=True)

Calculate the date range. If the `fixed_date` parameter is `false`, make today the latest date.

In [ ]:
date_range = (
    date.fromisoformat(start),
    min(date.today(), date.fromisoformat(end)) if fixed_date == 'false' else date.fromisoformat(end),
    date.today(),
)

Write the timestamp file

In [ ]:
with open(TARGET / 'timestamp.json', 'w') as f:
    json.dump(dict(zip(['start', 'end', 'updated'], (str(x) for x in date_range))), f)

Load programme data

In [ ]:
programme_data = ProgrammeSlice(date_range=date_range[0:2])

## Events & projects data

In [ ]:
events = (
    programme_data.project_breakdown
    .aggregate(['project_name', 'evaluation_category'], sum, 'events', field='events')
)

In [ ]:
project_summary = {
    'total': events.distinct('project_name').nrows(),
    'by_category': dict(events.aggregate('evaluation_category', len).records()),
    'names': list(events.cut('project_name').distinct().values('project_name')),
}

In [ ]:
events_summary = {
    'total': sum(events.values('events')),
    'by_category': dict(events.aggregate('evaluation_category', sum, 'events').records()),
    'by_project': {
        r[0]: {'category': r[1], 'count': r[2]}
        for r
        in events.records()
    },
}

In [ ]:
events_summary

In [ ]:
with open(TARGET / 'events.json', 'w') as f:
    json.dump(
        events_summary,
        f,
        indent=2
    )

with open(TARGET / 'projects.json', 'w') as f:
    json.dump(
        project_summary,
        f,
        indent=2
    )

## Audiences

In [ ]:
audience_data = (
    programme_data.project_breakdown
    .aggregate(['project_name', 'evaluation_category'], sum, 'audience', field='audience')
    .replace('audience', 0, None)
)

In [ ]:
project_to_spektrix_lookup = etl.fromcsv(PUBLISHED / 'lookups/project_to_spektrix_event.csv')

In [ ]:
tickets_sold = (
    etl
    .fromcsv(PUBLISHED / 'ticketing/tickets.csv')
    .selecteq('geography_type', 'oslaua')
    .rightjoin(
        etl
        .fromcsv(PUBLISHED / 'ticketing/event-instances.csv')
        .convert('start', etl.dateparser('%Y-%m-%d %H:%M:%S'))
        .selectrangeopen('start', date_range[0], date_range[1])
        .selectcontains('eventType', 'MainProgrammeEvent')
        .cut('instance_id', 'event_name', 'event_id')
    )
    .cut('event_name', 'event_id', 'geography_code', 'count_of_tickets')
    .convertnumbers()
    .selectnotnone('count_of_tickets')
    .aggregate(['event_name', 'event_id', 'geography_code'], sum, 'count_of_tickets', field='count_of_tickets')

    .leftjoin(project_to_spektrix_lookup.cutout('event_name'), lkey='event_id', rkey='event_id')

    .convert('event_name', lambda _, r: r.project_name, where=lambda r: r.project_name is not None, pass_row=True)
    .addfield('evaluation_category', 'In-person')
)

In [ ]:
tickets_scanned = dict(
    etl
    .fromcsv(PUBLISHED / 'manual/spektrix-sales-and-scans.csv')
    .convert('Event Date', etl.dateparser('%Y-%m-%d'))
    .selectrangeopen('Event Date', date_range[0], date_range[1])
    .convertnumbers()
    .aggregate(None, {
        'sold': ('Sold', sum),
        'scanned': ('Scanned', sum)
    })
    .melt(variables=['sold', 'scanned'])
    .records()
)

In [ ]:
ticketed_audience = {
    'sold': {
        'total': sum(tickets_sold.values('count_of_tickets')),
        'bradford': sum(tickets_sold.selecteq('geography_code', 'E08000032').values('count_of_tickets')),
        'by_project': dict(tickets_sold.aggregate('event_name', sum, 'count_of_tickets', field='count_of_tickets').records()),
        'by_geo': dict(tickets_sold.aggregate('geography_code', sum, 'count_of_tickets', field='count_of_tickets').records()),
    },
    'scanned': {
        'total': tickets_scanned
    }
}

Try to get audience for tickets that aren't matched

In [ ]:
audience_from_spektrix = tickets_sold.selectnone('project_name').aggregate('event_name', sum, 'count_of_tickets').addfield('evaluation_category', 'UNKNOWN').rename({
    'event_name': 'project_name',
    'value': 'audience'
})

In [ ]:
audience_plus_ticket_data = (
    audience_data
    .leftjoin(
        tickets_sold.aggregate(['project_name', 'evaluation_category'], sum, 'count_of_tickets', field='ticketed_audience'))
    .fillleft()
    .selectnotnone('audience')
)

In [ ]:
all_audience = etl.cat(
    audience_plus_ticket_data,
    audience_from_spektrix,
)

In [ ]:
audience = {
    'total': sum(all_audience.values('audience')),
    'by_category': dict(
        all_audience
        .aggregate(['evaluation_category'], sum, 'audience')
        .records()
    ),
    'by_project': {
        r[0]: {'category': r[1], 'count': r[2]}
        for r
        in all_audience
        .aggregate(['project_name', 'evaluation_category'], sum, 'audience')
        .records()
    },
}

In [ ]:
with open(TARGET / 'audience.json', 'w') as f:
    json.dump(
        audience | {
            'ticketed': ticketed_audience,
            'unticketed': None,
        },
        f,
        indent=2
    )

## Participants

### Programme

In [ ]:
programme_participants = (
    programme_data.project_breakdown
    .cut('project_name', 'evaluation_category', 'participants_community')
    .melt(['project_name', 'evaluation_category'])
    .selectnotnone('value')
    .aggregate(['project_name', 'evaluation_category', 'variable'], sum, 'value')
    .recast(samplesize=1_000_000)
    .selectgt('participants_community', 0)
)

### Cultural learning

In [ ]:
import petl as etl
from datetime import date
import ast

import petl_addons

def normalise_header(table):
    new_header = [c.lower().replace(' ', '_') for c in table.header()]
    return table.setheader(new_header)

cultural_learning_engagements = etl.cat(
    etl
    .fromcsv(PUBLISHED / 'cultural-learning/engagements.csv' )
    .convert(['date', 'month'], date.fromisoformat)
    .selectnotnone('date')
    .selectrangeopen('date', *date_range[:2])
    .convertnumbers()
    .convert('organisation_ids', ast.literal_eval),

    etl
    .fromcsv(PUBLISHED / 'manual/manual-events.csv')
    .cut('Date', 'Project', 'Evaluation category', 'participants_schools')
    .apply(normalise_header)
    .rename({
        'project': 'project_name',
        'participants_schools': 'participants'
    })
    .selectne('participants', '')
    .convertnumbers()
    .convert('date', date.fromisoformat)
    .selectrangeopen('date', *date_range[:2])
    .addfield('organisation_ids', [])
)

cultural_learning_settings = (
    etl
    .fromcsv(PUBLISHED / 'programme/venues.csv')
    .selecteq('cultural_learning', 'True')
)

def distinct(items):
    cl_orgs = cultural_learning_settings.values('id')
    return len({i for s in items for i in s if i in cl_orgs})

cultural_learning_summary = (
    cultural_learning_engagements
    .aggregate(None, {
        'participants': ('participants', sum),
        'schools_engaged': ('organisation_ids', distinct)
    })
    .dicts()[0]
)

Report by org type

In [ ]:
cultural_learning_settings.aggregate('Org/Venue Type', len)

### Volunteers

In [ ]:
volunteer_signups = (
    etl
    .fromcsv(PUBLISHED / 'volunteers/checkpoint-updates.csv')
    .convert('date', etl.dateparser('%Y-%m-%d'))
    .selectle('date', date_range[1])
    .convertnumbers(strict=False)
)

In [ ]:
# volunteer_shifts = (
#     etl
#     .fromcsv(PUBLISHED / 'volunteers/shifts.csv')
#     .convert('date', etl.dateparser('%Y-%m-%d'))
#     .selecteq('type', 'BD25 Event')
#     .selectrangeopen('date', date_range[0], date_range[1])
#     .convertnumbers(strict=False)
# )

In [ ]:
volunteer_summary = {
    'total': sum(
        volunteer_signups
        .selectin('checkpoint',  ["1. Monitoring & Evaluation", "2. Sign Up to Induction", "3. Fully Inducted Volunteers"])
        .values('count')
    ),
    'people': {
        'checkpoints': dict(volunteer_signups.aggregate('checkpoint', sum, 'count').records()),
    },
    # 'events': {
    #     'count': volunteer_shifts.cut('rosterfy_event_name').distinct().nrows(),
    #     'names': list(volunteer_shifts.cut('rosterfy_event_name').distinct().values('rosterfy_event_name')),
    # },
    # 'shifts': {
    #     'count': volunteer_shifts.nrows(),
    #     'attended': sum(volunteer_shifts.values('attended')),
    #     'hours': sum(volunteer_shifts.values('hours')),
    # },
}

### Other figures

In [ ]:
other_figures = etl.fromcsv(PUBLISHED / 'manual/manual-other-figures.csv').selecteq('dashboard key', target).convertnumbers()

In [ ]:
dict(other_figures.selecteq('variable', 'training_and_skills_development_participants').cut('source', 'value').rename('source', 'project_name').aggregate('project_name', sum, 'value', field='count').records())

In [ ]:
with open(TARGET / 'otherFigures.json', 'w') as f:
    json.dump(
        other_figures.aggregate('variable', sum, 'value').transpose().cutout('variable').dicts()[0],
        f,
        indent=2
    )

### Summary

In [ ]:
def summarise_by_type(table):
    return {
        'count': sum(table.values('count')),
        'by_project': {
            r.project_name: { 'count': r['count'], 'category': r.get('evaluation_category', None) }
            for r in
            table.selectgt('count', 0).records()
        },
    }

by_type = {
    'community_participants': summarise_by_type(programme_participants
        .rename('participants_community', 'count')
    ),
    'training_and_skills_development_participants': summarise_by_type(
        other_figures
        .selecteq('variable', 'training_and_skills_development_participants')
        .cut('source', 'value')
        .rename('source', 'project_name')
        .aggregate('project_name', sum, 'value', field='count')   
    ),
    'cultural_learning_participants': summarise_by_type(
        cultural_learning_engagements.aggregate(['project_name', 'evaluation_category'], sum, 'participants', field='count')
    )
}

In [ ]:
by_type

In [ ]:
participant_summary = {
    'total': sum(programme_participants.values('participants_community'))
        + cultural_learning_summary['participants']
        + volunteer_summary['total']
        + sum(other_figures.selectcontains('variable', '_participants').values('value')),

    'by_type': by_type,

    'cultural_learning': cultural_learning_summary,
    'volunteers': volunteer_summary,
}

In [ ]:
with open(TARGET / 'participants.json', 'w') as f:
    json.dump(
        participant_summary,
        f,
        indent=2
    )

## Other figures